# Investigate `B10_111120` Experiment
From the `orientation_representation_figures` notebook, experiment `B10_111120` was identified as containing NaNs in its HDF5 file. We will investigate the nature and pervasiveness of those NaNs.

## Imports

In [1]:
%cd ../../

/home/jovyan/work/task-priors


In [2]:
import os
import h5py

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm

from src.data_access import(
DATASET1, 
PERMISSABLE_FIT_TYPES,
load_tables, 
retrieve_spike_trace,
)

In [3]:
tables = load_tables()
df_celltable, df_trialtable = tables

/home/jovyan/work/task-priors/src/data_access.py:21: DtypeWarning: Columns (6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  df_celltable = pd.read_csv(DATASET['CellTable_path'])


In [4]:
experiment_id = 'B10_111120'

## Investigation

In [5]:
def retrieve_admissible_neurons(relevant_experiments, loaded_tables, fit_types=PERMISSABLE_FIT_TYPES):
    assert isinstance(relevant_experiments, list), 'relevant_experiments is not list'
    
    df_celltable, df_trialtable = loaded_tables
    admissible_neurons = df_celltable[
        (df_celltable['Experiment'].isin(relevant_experiments)) &\
        (df_celltable['Best_Fit_spikes_2'].isin(fit_types))
    ]
    return admissible_neurons[['Cell','Experiment','Best_Fit_spikes_2','Pref_Orientation_spikes_2']]

In [6]:
df_celltable_experiment = retrieve_admissible_neurons([experiment_id,], tables)
df_trialtable_experiment = df_trialtable[
    (df_trialtable['Experiment'] == experiment_id) &\
    (df_trialtable['Block'] == 'Visual')
]

In [7]:
stop_outer_loop = False

for index, trial_row in tqdm(df_trialtable_experiment.iterrows()):
    for index, cell_row in df_celltable_experiment.iterrows():
        spike_trace = retrieve_spike_trace(experiment_id, trial_row['Trial'], cell_row['Cell'])
        
        if np.isnan(spike_trace).any():
            stop_outer_loop = True
            break
    
    if stop_outer_loop:
        break

0it [00:00, ?it/s]


In [8]:
spike_trace

array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, na

In [9]:
cell_row

Cell                                          1543
Experiment                              B10_111120
Best_Fit_spikes_2            circular_gaussian_360
Pref_Orientation_spikes_2                    133.0
Name: 255, dtype: object

In [10]:
trial_row

Trial                                                                  121
Experiment                                                      B10_111120
Behav_Cond                                                              D3
Block                                                               Visual
Visual_Stim                                                             45
Auditory_Stim                                                           -1
Contrast                                                              0.25
Response                                                                 1
Outcome                                                                Hit
Lick_trace               0x0001494D000000000E000000981B0400060000000800...
Lick_dt                                                             0.0004
Locomotion_trace         0x0001494D000000000E000000B8060000060000000800...
Locomotion_mean_01                                                 1.45134
Pupil_Area_trace         

In [11]:
hdf5_path = os.path.join(
    DATASET1['CellTrialTabel_csv_path'], 
    experiment_id[:3], 
    f"{experiment_id}.h5",
)

cell_index = 1543 - 1 # Very important to decrease by 1 for proper indexing
with h5py.File(hdf5_path, 'r') as hdf:
    trial_name = f'trial_{121}'
    trial_matrix = hdf[trial_name][:]
    cell_trial_spike_train = trial_matrix[:, cell_index]

In [12]:
np.where(np.isnan(trial_matrix[0,:]))

(array([  35,  244,  411,  444,  504,  565,  573,  986, 1084, 1085, 1086,
        1087, 1088, 1089, 1090, 1091, 1241, 1512, 1513, 1514, 1515, 1516,
        1517, 1518, 1519, 1520, 1521, 1522, 1523, 1524, 1525, 1526, 1527,
        1528, 1529, 1530, 1531, 1532, 1533, 1534, 1535, 1536, 1537, 1538,
        1539, 1540, 1541, 1542, 1543, 1544, 1545, 1546, 1547, 1584, 1585,
        1586, 1587, 1588, 1589, 1590, 1591, 1592, 1593, 1594, 1595, 1596,
        1597, 1598, 1599, 1600, 1601, 1602, 1603, 1604, 1605, 1606, 1607,
        1608, 1609, 1610, 1611, 1612, 1613, 1614, 1615, 1616, 1617, 1618,
        1619, 1620, 1621, 1622, 1623, 1624, 1625, 1626, 1627, 1628, 1629,
        1630, 1631, 1632, 1633, 1634, 1635, 1636, 1637, 1638, 1639, 1640,
        1641, 1642, 1643, 1644, 1645, 1646, 1647, 1648, 1649, 1650, 1651,
        1652, 1653, 1654, 1655, 1656, 1657, 1658, 1659, 1660, 1661, 1662,
        1663, 1664, 1665, 1666, 1667, 1668, 1669, 1670, 1671, 1672, 1673,
        1674, 1675, 1676, 1677, 1678, 

In [13]:
with h5py.File(hdf5_path, 'r') as hdf:
    trial_name = f'trial_{120}'
    trial_matrix = hdf[trial_name][:]

nan_cell_inx = np.where(np.isnan(trial_matrix[0,:]))[0]
nan_cell_inx

array([  35,  244,  411,  444,  504,  565,  573,  986, 1084, 1085, 1086,
       1087, 1088, 1089, 1090, 1091, 1241, 1512, 1513, 1514, 1515, 1516,
       1517, 1518, 1519, 1520, 1521, 1522, 1523, 1524, 1525, 1526, 1527,
       1528, 1529, 1530, 1531, 1532, 1533, 1534, 1535, 1536, 1537, 1538,
       1539, 1540, 1541, 1542, 1543, 1544, 1545, 1546, 1547, 1584, 1585,
       1586, 1587, 1588, 1589, 1590, 1591, 1592, 1593, 1594, 1595, 1596,
       1597, 1598, 1599, 1600, 1601, 1602, 1603, 1604, 1605, 1606, 1607,
       1608, 1609, 1610, 1611, 1612, 1613, 1614, 1615, 1616, 1617, 1618,
       1619, 1620, 1621, 1622, 1623, 1624, 1625, 1626, 1627, 1628, 1629,
       1630, 1631, 1632, 1633, 1634, 1635, 1636, 1637, 1638, 1639, 1640,
       1641, 1642, 1643, 1644, 1645, 1646, 1647, 1648, 1649, 1650, 1651,
       1652, 1653, 1654, 1655, 1656, 1657, 1658, 1659, 1660, 1661, 1662,
       1663, 1664, 1665, 1666, 1667, 1668, 1669, 1670, 1671, 1672, 1673,
       1674, 1675, 1676, 1677, 1678, 1679, 1680, 16

In [14]:
nan_cell_inx.shape

(186,)

In [15]:
df_HDF5_nan = df_celltable_experiment[df_celltable_experiment['Cell'].isin(list(nan_cell_inx + 1))]
df_HDF5_nan

,Cell,Experiment,Best_Fit_spikes_2,Pref_Orientation_spikes_2
255,1543,B10_111120,circular_gaussian_360,133.0
259,1595,B10_111120,circular_gaussian_360,322.0
260,1605,B10_111120,circular_gaussian_360,125.0
3235,1085,B10_111120,direction_selective_circular_gaussian,297.0
3236,1086,B10_111120,direction_selective_circular_gaussian,270.0
...,...,...,...,...
3903,1717,B10_111120,direction_selective_circular_gaussian,270.0
3904,1718,B10_111120,direction_selective_circular_gaussian,203.0
3905,1719,B10_111120,direction_selective_circular_gaussian,270.0
3906,1720,B10_111120,direction_selective_circular_gaussian,270.0


In [16]:
df_tau_nan = df_celltable[
    (df_celltable['Experiment'] == experiment_id) &\
    (df_celltable['DeconvA_Tau'].isna()) &\
    (df_celltable['Best_Fit_spikes_2'].isin(PERMISSABLE_FIT_TYPES))
]
df_tau_nan

,Cell,Experiment,Zscore,DeconvA_Tau,nSpikeTC,DeconvCorr,Tuning_Curve_spikes_1,Best_Fit_spikes_1,Pref_Orientation_spikes_1,Tuning_Width_spikes_1,...,Pref_Orientation_AuC_2,Tuning_Width_AuC_2,DSI_AuC_2,OSI_AuC_2,isCell,nPix,spikeProb_TCTrials,roundness,radius,magFactor
255,1543,B10_111120,15.587093,NaN,NaN,NaN,0x0001494D000000000E000000700B0000060000000800...,circular_gaussian_360,189.0,55.867603,...,NaN,NaN,NaN,NaN,0.846088,72,0.00,0.436709,7.034781,0.5
259,1595,B10_111120,17.911964,NaN,NaN,NaN,0x0001494D000000000E000000700B0000060000000800...,circular_gaussian_360,341.0,18.885481,...,NaN,NaN,NaN,NaN,0.601356,27,0.75,0.052314,12.606230,0.5
260,1605,B10_111120,15.463325,NaN,NaN,NaN,0x0001494D000000000E000000700B0000060000000800...,circular_gaussian_180,123.0,27.108452,...,NaN,NaN,NaN,NaN,0.865884,47,0.75,0.693548,4.472136,0.5
3235,1085,B10_111120,19.805390,NaN,NaN,NaN,0x0001494D000000000E000000700B0000060000000800...,circular_gaussian_180,82.0,43.329387,...,NaN,NaN,NaN,NaN,0.879519,86,0.00,0.383178,8.294025,0.5
3236,1086,B10_111120,13.727775,NaN,NaN,NaN,0x0001494D000000000E000000700B0000060000000800...,constant,NaN,NaN,...,NaN,NaN,NaN,NaN,0.894700,49,0.00,0.723077,4.498529,0.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3903,1717,B10_111120,19.151445,NaN,NaN,NaN,0x0001494D000000000E000000700B0000060000000800...,circular_gaussian_180,42.0,56.584244,...,NaN,NaN,NaN,NaN,0.317581,19,0.00,0.237500,5.054461,0.5
3904,1718,B10_111120,18.550997,NaN,NaN,NaN,0x0001494D000000000E000000700B0000060000000800...,direction_selective_circular_gaussian,278.0,41.860722,...,NaN,NaN,NaN,NaN,0.734548,15,0.00,0.144330,5.562072,0.5
3905,1719,B10_111120,25.525730,NaN,NaN,NaN,0x0001494D000000000E000000700B0000060000000800...,circular_gaussian_360,213.0,44.908092,...,NaN,NaN,NaN,NaN,0.917341,30,0.00,0.109375,9.055385,0.5
3906,1720,B10_111120,12.218348,NaN,NaN,NaN,0x0001494D000000000E000000700B0000060000000800...,circular_gaussian_180,121.0,22.069376,...,NaN,NaN,NaN,NaN,0.479117,53,0.00,0.320513,7.078120,0.5


In [17]:
np.array_equal(df_HDF5_nan['Cell'].unique(), df_tau_nan['Cell'].unique())

True

In [18]:
df_celltable[
    (df_celltable['DeconvA_Tau'].isna()) &\
    (df_celltable['Best_Fit_spikes_2'].isin(PERMISSABLE_FIT_TYPES))
]

,Cell,Experiment,Zscore,DeconvA_Tau,nSpikeTC,DeconvCorr,Tuning_Curve_spikes_1,Best_Fit_spikes_1,Pref_Orientation_spikes_1,Tuning_Width_spikes_1,...,Pref_Orientation_AuC_2,Tuning_Width_AuC_2,DSI_AuC_2,OSI_AuC_2,isCell,nPix,spikeProb_TCTrials,roundness,radius,magFactor
255,1543,B10_111120,15.587093,NaN,NaN,NaN,0x0001494D000000000E000000700B0000060000000800...,circular_gaussian_360,189.0,55.867603,...,NaN,NaN,NaN,NaN,0.846088,72,0.00,0.436709,7.034781,0.5
259,1595,B10_111120,17.911964,NaN,NaN,NaN,0x0001494D000000000E000000700B0000060000000800...,circular_gaussian_360,341.0,18.885481,...,NaN,NaN,NaN,NaN,0.601356,27,0.75,0.052314,12.606230,0.5
260,1605,B10_111120,15.463325,NaN,NaN,NaN,0x0001494D000000000E000000700B0000060000000800...,circular_gaussian_180,123.0,27.108452,...,NaN,NaN,NaN,NaN,0.865884,47,0.75,0.693548,4.472136,0.5
3235,1085,B10_111120,19.805390,NaN,NaN,NaN,0x0001494D000000000E000000700B0000060000000800...,circular_gaussian_180,82.0,43.329387,...,NaN,NaN,NaN,NaN,0.879519,86,0.00,0.383178,8.294025,0.5
3236,1086,B10_111120,13.727775,NaN,NaN,NaN,0x0001494D000000000E000000700B0000060000000800...,constant,NaN,NaN,...,NaN,NaN,NaN,NaN,0.894700,49,0.00,0.723077,4.498529,0.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3903,1717,B10_111120,19.151445,NaN,NaN,NaN,0x0001494D000000000E000000700B0000060000000800...,circular_gaussian_180,42.0,56.584244,...,NaN,NaN,NaN,NaN,0.317581,19,0.00,0.237500,5.054461,0.5
3904,1718,B10_111120,18.550997,NaN,NaN,NaN,0x0001494D000000000E000000700B0000060000000800...,direction_selective_circular_gaussian,278.0,41.860722,...,NaN,NaN,NaN,NaN,0.734548,15,0.00,0.144330,5.562072,0.5
3905,1719,B10_111120,25.525730,NaN,NaN,NaN,0x0001494D000000000E000000700B0000060000000800...,circular_gaussian_360,213.0,44.908092,...,NaN,NaN,NaN,NaN,0.917341,30,0.00,0.109375,9.055385,0.5
3906,1720,B10_111120,12.218348,NaN,NaN,NaN,0x0001494D000000000E000000700B0000060000000800...,circular_gaussian_180,121.0,22.069376,...,NaN,NaN,NaN,NaN,0.479117,53,0.00,0.320513,7.078120,0.5


In [19]:
df_celltable[
    (~df_celltable['DeconvA_Tau'].isna()) &\
    (df_celltable['Best_Fit_spikes_2'].isin(PERMISSABLE_FIT_TYPES))
]

,Cell,Experiment,Zscore,DeconvA_Tau,nSpikeTC,DeconvCorr,Tuning_Curve_spikes_1,Best_Fit_spikes_1,Pref_Orientation_spikes_1,Tuning_Width_spikes_1,...,Pref_Orientation_AuC_2,Tuning_Width_AuC_2,DSI_AuC_2,OSI_AuC_2,isCell,nPix,spikeProb_TCTrials,roundness,radius,magFactor
0,1,B10_110920,22.635183,0x0001494D000000000E00000038000000060000000800...,4386.0,0.987410,0x0001494D000000000E000000700B0000060000000800...,circular_gaussian_360,185.0,17.773541,...,NaN,NaN,NaN,NaN,0.349529,60,0.100000,0.027616,25.668601,0.5
1,2,B10_110920,15.369622,0x0001494D000000000E00000038000000060000000800...,2901.0,0.984389,0x0001494D000000000E000000700B0000060000000800...,circular_gaussian_360,345.0,25.697329,...,NaN,NaN,NaN,NaN,0.348195,224,0.100000,0.580381,10.839231,0.5
2,3,B10_110920,17.427969,0x0001494D000000000E00000038000000060000000800...,3618.0,0.987856,0x0001494D000000000E000000700B0000060000000800...,direction_selective_circular_gaussian,345.0,17.029670,...,NaN,NaN,NaN,NaN,0.741653,143,1.000000,0.660099,8.062258,0.5
3,4,B10_110920,32.957703,0x0001494D000000000E00000038000000060000000800...,9074.0,0.986769,0x0001494D000000000E000000700B0000060000000800...,circular_gaussian_180,92.0,16.415984,...,NaN,NaN,NaN,NaN,0.832610,55,0.578947,0.625000,5.288874,0.5
4,5,B10_110920,19.673850,0x0001494D000000000E00000038000000060000000800...,2459.0,0.964936,0x0001494D000000000E000000700B0000060000000800...,direction_selective_circular_gaussian,89.0,16.171344,...,NaN,NaN,NaN,NaN,0.948684,20,0.157895,0.169492,6.203659,0.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81912,1043,F19_42521_,6.477872,0x0001494D000000000E00000038000000060000000800...,604.0,0.752171,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.488260,49,0.500000,0.105727,12.047791,0.5
81913,1044,F19_42521_,5.654099,0x0001494D000000000E00000038000000060000000800...,373.0,0.735433,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.560942,48,0.400000,0.696970,4.563439,0.5
81915,1046,F19_42521_,4.753120,0x0001494D000000000E00000038000000060000000800...,118.0,0.594590,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.345374,70,0.200000,0.814815,5.099020,0.5
81916,1047,F19_42521_,7.626988,0x0001494D000000000E00000038000000060000000800...,178.0,0.731369,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.546211,41,0.050000,0.075435,12.927577,0.5
